In [20]:
import random
from lang import load_data_int_seq
from utils import eq_encoder, is_eq_valid
from typing import List
from models.rl.env import IntegerSequenceEnv, get_current_position, encode_with_lang, decode_with_lang
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

In [22]:
def compare_sequences(output_sequence: List[int], target_sequence: List[int]) -> float:
    magnitude: float = 0.0
    for x, y in enumerate(zip(output_sequence, target_sequence)):
        magnitude += abs(x - y)#**2
    # magnitude /= len(norm_target_seq)
    return magnitude


In [23]:
def evaluate_candidate_eq(candidate: str, int_seq: List[int]) -> float:
    if is_eq_valid(candidate) == False:
        return 99.0

    output_sequence = eq_to_seq(candidate, 9)

    if np.count_nonzero(output_sequence) < 1:
        return max_penalty_magnitude

    return compare_sequences(output_sequence, int_seq)

In [24]:
env = IntegerSequenceEnv(train[0][0], 9, input_lang, output_lang, evaluate_candidate_eq)

In [25]:
for _ in range(0, 8):
    env.step(1)

In [26]:
env.step(1)

(([3, 3, 3, 3, 3, 3, 3, 3, 3], [2, 3, 4, 5, 6, 7, 8, 9]), 99.0, True)